# Adverse Events Analysis Example

This notebook demonstrates how to perform adverse events analysis and incorporate disutilities in health economic evaluation.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Define adverse events analysis functions
def calculate_adverse_event_disutilities(
    strategy,
    adverse_events,
    severity_weights,
    duration_weights,
    base_utility=0.85
):
    """
    Calculate utility reductions due to adverse events.
    
    Args:
        strategy: Treatment strategy name
        adverse_events: Dictionary of adverse events with frequencies
        severity_weights: Dictionary of severity weights (0-1)
        duration_weights: Dictionary of duration weights (0-1)
        base_utility: Base utility when healthy
        
    Returns:
        Dictionary with AE-adjusted utilities
    """
    total_disutility = 0
    
    for ae, frequency in adverse_events.items():
        severity_weight = severity_weights.get(ae, 0.2)
        duration_weight = duration_weights.get(ae, 0.1)
        
        # Calculate disutility contribution for this AE
        ae_disutility = frequency * severity_weight * duration_weight
        total_disutility += ae_disutility
    
    # Adjust base utility
    adjusted_utility = base_utility - total_disutility
    
    # Ensure utility doesn't go below 0
    adjusted_utility = max(0, adjusted_utility)
    
    return {
        'strategy': strategy,
        'base_utility': base_utility,
        'total_disutility': total_disutility,
        'adjusted_utility': adjusted_utility,
        'relative_utility': adjusted_utility / base_utility if base_utility > 0 else 0,
        'adverse_events': adverse_events,
        'adverse_event_details': {
            ae: {
                'frequency': freq,
                'severity_weight': severity_weights.get(ae, 0.2),
                'duration_weight': duration_weights.get(ae, 0.1),
                'disutility_contribution': freq * severity_weights.get(ae, 0.2) * duration_weights.get(ae, 0.1)
            } for ae, freq in adverse_events.items()
        }
    }

def incorporate_ae_effects(cost, effect, ae_adjusted_utility, base_utility=0.85):
    """
    Adjust cost-effectiveness considering adverse events.
    
    Args:
        cost: Original cost
        effect: Original effect (e.g., QALYs)
        ae_adjusted_utility: Utility adjusted for adverse events
        base_utility: Base utility without AEs
        
    Returns:
        Dictionary with AE-adjusted outcomes
    """
    # Calculate adjustment factor
    utility_ratio = ae_adjusted_utility / base_utility if base_utility > 0 else 1
    
    # Adjust the effect based on utility reduction
    adjusted_effect = effect * utility_ratio
    
    # Calculate adjusted ICER if needed
    if effect > 0:
        adjusted_icer = cost / adjusted_effect
        original_icer = cost / effect
    else:
        adjusted_icer = np.inf
        original_icer = np.inf
    
    return {
        'original_cost': cost,
        'original_effect': effect,
        'original_icer': original_icer,
        'ae_adjusted_cost': cost,  # Cost doesn't change due to AEs
        'ae_adjusted_effect': adjusted_effect,
        'ae_adjusted_icer': adjusted_icer,
        'utility_ratio': utility_ratio,
        'effect_reduction': effect - adjusted_effect
    }

In [ ]:
# Define adverse event data for each strategy
strategies = ['ECT', 'IV-KA', 'PO-KA']
ae_data = {
    'ECT': {
        'cognitive_impairment': 0.30,  # 30% incidence
        'memory_loss': 0.25,          # 25% incidence
        'cardiac_issues': 0.05,       # 5% incidence
        'muscle_soreness': 0.40,      # 40% incidence
    },
    'IV-KA': {
        'dissociation': 0.60,         # 60% incidence
        'elevation_bp': 0.45,         # 45% incidence
        'anxiety': 0.35,              # 35% incidence
        'bladder_issues': 0.05,       # 5% incidence (less than ECT)
    },
    'PO-KA': {
        'nausea': 0.50,              # 50% incidence
        'dizziness': 0.40,           # 40% incidence
        'fatigue': 0.35,             # 35% incidence
        'mild_anxiety': 0.60,        # 60% incidence
    }
}

# Define severity and duration weights
severity_weights = {
    'cognitive_impairment': 0.4,
    'memory_loss': 0.5,
    'cardiac_issues': 0.8,
    'muscle_soreness': 0.1,
    'dissociation': 0.3,
    'elevation_bp': 0.2,
    'anxiety': 0.2,
    'bladder_issues': 0.4,
    'nausea': 0.15,
    'dizziness': 0.1,
    'fatigue': 0.1,
    'mild_anxiety': 0.1,
}

duration_weights = {
    'cognitive_impairment': 0.7,  # Longer-term effect
    'memory_loss': 0.8,           # Potentially permanent
    'cardiac_issues': 0.3,        # Short-term if treated
    'muscle_soreness': 0.2,       # Short-term
    'dissociation': 0.5,          # Moderate duration
    'elevation_bp': 0.1,          # Short-term
    'anxiety': 0.4,               # Variable duration
    'bladder_issues': 0.6,        # Moderate duration
    'nausea': 0.3,                # Short-term
    'dizziness': 0.2,             # Short-term
    'fatigue': 0.4,               # Variable duration
    'mild_anxiety': 0.3,          # Short to medium-term
}

# Calculate AE-adjusted utilities
ae_results = []
for strategy in strategies:
    result = calculate_adverse_event_disutilities(
        strategy,
        ae_data[strategy],
        severity_weights,
        duration_weights
    )
    ae_results.append(result)

ae_df = pd.DataFrame([
    {
        'strategy': r['strategy'],
        'base_utility': r['base_utility'],
        'total_disutility': r['total_disutility'],
        'adjusted_utility': r['adjusted_utility'],
        'relative_utility': r['relative_utility']
    } for r in ae_results
])

print(ae_df)

# Calculate AE-adjusted effects for example cost-effect data
cost_data = [4500, 7000, 5500]  # Example costs
effect_data = [0.58, 0.78, 0.70]  # Example effects (QALYs)

ae_adjusted_results = []
for i, strategy in enumerate(strategies):
    adj = incorporate_ae_effects(
        cost_data[i],
        effect_data[i],
        ae_df[ae_df['strategy'] == strategy]['adjusted_utility'].iloc[0]
    )
    adj['strategy'] = strategy
    ae_adjusted_results.append(adj)

ae_effect_df = pd.DataFrame(ae_adjusted_results)
print(f"\nAE-Adjusted Cost-Effectiveness:")
print(ae_effect_df[['strategy', 'original_effect', 'ae_adjusted_effect', 'original_icer', 'ae_adjusted_icer']])

In [ ]:
# Visualize adverse events analysis
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

# 1. Base vs Adjusted Utility Comparison
x = np.arange(len(ae_df))
width = 0.35

ax[0, 0].bar(x - width/2, ae_df['base_utility'], width, label='Base Utility', alpha=0.7, color='lightblue')
ax[0, 0].bar(x + width/2, ae_df['adjusted_utility'], width, label='AE-Adjusted Utility', alpha=0.7, color='salmon')
ax[0, 0].set_xlabel('Strategy')
ax[0, 0].set_ylabel('Utility')
ax[0, 0].set_title('Base vs AE-Adjusted Utilities')
ax[0, 0].set_xticks(x)
ax[0, 0].set_xticklabels(ae_df['strategy'])
ax[0, 0].legend()
ax[0, 0].grid(True, alpha=0.3)

# 2. Total Disutility by Strategy
colors = ['blue' if s == 'ECT' else 'green' if s == 'IV-KA' else 'orange' for s in ae_df['strategy']]
ax[0, 1].bar(ae_df['strategy'], ae_df['total_disutility'], color=colors, alpha=0.7)
ax[0, 1].set_ylabel('Total Disutility')
ax[0, 1].set_title('Total Disutility Due to Adverse Events by Strategy')
ax[0, 1].grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(ae_df['total_disutility']):
    ax[0, 1].text(i, v + max(ae_df['total_disutility'])*0.01, 
                 f'{v:.3f}', ha='center', va='bottom')

# 3. Effect Reduction due to AEs
colors = ['blue' if s == 'ECT' else 'green' if s == 'IV-KA' else 'orange' for s in ae_effect_df['strategy']]
ax[1, 0].bar(ae_effect_df['strategy'], ae_effect_df['effect_reduction'], color=colors, alpha=0.7)
ax[1, 0].set_ylabel('Effect Reduction (QALYs)')
ax[1, 0].set_title('Reduction in Health Effect Due to Adverse Events')
ax[1, 0].grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate(ae_effect_df['effect_reduction']):
    ax[1, 0].text(i, v + max(ae_effect_df['effect_reduction'])*0.01, 
                 f'{v:.3f}', ha='center', va='bottom')

# 4. ICER Comparison (Original vs AE-Adjusted)
ax[1, 1].bar(ae_effect_df['strategy'], ae_effect_df['original_icer'], 
            width=0.4, label='Original ICER', alpha=0.7, color='lightblue')
ax[1, 1].bar(ae_effect_df['strategy'], ae_effect_df['ae_adjusted_icer'], 
            width=0.4, label='AE-Adjusted ICER', alpha=0.7, color='salmon')
ax[1, 1].set_ylabel('ICER ($/QALY)')
ax[1, 1].set_title('ICER Comparison: Original vs AE-Adjusted')
ax[1, 1].legend()
ax[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Detail of adverse event contributions
plt.figure(figsize=(14, 8))

# Prepare data for stacked bar chart
ae_details_list = []
for i, result in enumerate(ae_results):
    strategy = result['strategy']
    for ae, details in result['adverse_event_details'].items():
        ae_details_list.append({
            'strategy': strategy,
            'adverse_event': ae,
            'frequency': details['frequency'],
            'severity_weight': details['severity_weight'],
            'duration_weight': details['duration_weight'],
            'disutility_contribution': details['disutility_contribution']
        })

ae_details_df = pd.DataFrame(ae_details_list)

# Pivot to create a stacked bar chart
ae_pivot = ae_details_df.pivot_table(
    values='disutility_contribution', 
    index='strategy', 
    columns='adverse_event', 
    fill_value=0
)

# Create stacked bar chart
ae_pivot.plot(kind='bar', stacked=True, ax=plt.gca(), colormap='tab20')
plt.ylabel('Disutility Contribution')
plt.title('Adverse Event Contributions to Total Disutility by Strategy')
plt.legend(title='Adverse Event', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, axis='y', alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Print detailed breakdown
print("ADVERSE EVENT BREAKDOWN BY STRATEGY")
print("="*60)
for result in ae_results:
    print(f"\n{result['strategy']}:\n" + "-"*30)
    print(f"  Base Utility: {result['base_utility']:.3f}")
    print(f"  Total Disutility: {result['total_disutility']:.3f}")
    print(f"  Adjusted Utility: {result['adjusted_utility']:.3f}")
    print(f"  Relative Utility: {result['relative_utility']:.3f} ({result['relative_utility']*100:.1f}%)")
    
    print("  Adverse Event Contributions:")
    for ae, details in result['adverse_event_details'].items():
        print(f"    - {ae}: Frequency={details['frequency']:.2f}, \
              Severity={details['severity_weight']:.2f}, Duration={details['duration_weight']:.2f}, \
              Contribution={details['disutility_contribution']:.4f}")

In [ ]:
# Perform sensitivity analysis on severity weights
severity_multipliers = [0.5, 0.75, 1.0, 1.25, 1.5]  # Different severity weighting scenarios
sensitivity_results = []

for mult in severity_multipliers:
    adjusted_severity = {k: v * mult for k, v in severity_weights.items()}
    
    for strategy in strategies:
        result = calculate_adverse_event_disutilities(
            strategy,
            ae_data[strategy],
            adjusted_severity,
            duration_weights
        )
        
        sensitivity_results.append({
            'strategy': strategy,
            'severity_multiplier': mult,
            'adjusted_utility': result['adjusted_utility'],
            'total_disutility': result['total_disutility'],
            'relative_utility': result['relative_utility']
        })

sens_df = pd.DataFrame(sensitivity_results)

# Plot sensitivity analysis
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Adjusted utility vs severity multiplier
for strategy in strategies:
    strat_data = sens_df[sens_df['strategy'] == strategy]
    ax[0].plot(strat_data['severity_multiplier'], strat_data['adjusted_utility'], 
               label=strategy, marker='o', linewidth=2, markersize=6)

ax[0].set_xlabel('Severity Weight Multiplier')
ax[0].set_ylabel('AE-Adjusted Utility')
ax[0].set_title('Sensitivity Analysis: Adjusted Utility vs Severity Weights')
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Total disutility vs severity multiplier
for strategy in strategies:
    strat_data = sens_df[sens_df['strategy'] == strategy]
    ax[1].plot(strat_data['severity_multiplier'], strat_data['total_disutility'], 
               label=strategy, marker='s', linewidth=2, markersize=6)

ax[1].set_xlabel('Severity Weight Multiplier')
ax[1].set_ylabel('Total Disutility')
ax[1].set_title('Sensitivity Analysis: Total Disutility vs Severity Weights')
ax[1].legend()
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate Quality-Adjusted Life Years (QALYs) loss due to AEs
time_horizon = 5  # 5-year time horizon

qaly_losses = []
for _, row in ae_effect_df.iterrows():
    # Calculate total QALYs lost due to AEs over the time horizon
    effect_red = row['effect_reduction']
    qaly_lost = effect_red * time_horizon
    
    # Calculate monetary value of QALYs lost at WTP threshold
    wtp_threshold = 50000  # $50,000 per QALY
    monetary_value_lost = qaly_lost * wtp_threshold
    
    qaly_losses.append({
        'strategy': row['strategy'],
        'qalys_lost_5y': qaly_lost,
        'monetary_value_lost': monetary_value_lost,
        'cost_per_qaly_lost': row['original_cost'] / effect_red if effect_red > 0 else float('inf')
    })

qaly_loss_df = pd.DataFrame(qaly_losses)
print("QALY LOSSES DUE TO ADVERSE EVENTS")
print("="*50)
print(qaly_loss_df)

# Visualize QALY losses
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# QALYs lost
colors = ['blue' if s == 'ECT' else 'green' if s == 'IV-KA' else 'orange' for s in qaly_loss_df['strategy']]
ax[0].bar(qaly_loss_df['strategy'], qaly_loss_df['qalys_lost_5y'], color=colors, alpha=0.7)
ax[0].set_ylabel('QALYs Lost (5-year horizon)')
ax[0].set_title('Quality-Adjusted Life Years Lost Due to Adverse Events')
ax[0].grid(True, alpha=0.3)

# Monetary value lost
ax[1].bar(qaly_loss_df['strategy'], qaly_loss_df['monetary_value_lost'], color=colors, alpha=0.7)
ax[1].set_ylabel('Monetary Value Lost ($AUD at $50k/QALY)')
ax[1].set_title('Monetary Value of QALY Losses Due to Adverse Events')
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Generate recommendations considering adverse events
print("ADVERSE EVENTS ANALYSIS RECOMMENDATIONS")
print("="*60)
for _, row in ae_effect_df.iterrows():
    ae_row = ae_df[ae_df['strategy'] == row['strategy']].iloc[0]
    loss_row = qaly_loss_df[qaly_loss_df['strategy'] == row['strategy']].iloc[0]
    
    print(f"\n{row['strategy']}:\n" + "-"*30)
    print(f"  Original ICER: ${row['original_icer']:,.0f}/QALY")
    print(f"  AE-Adjusted ICER: ${row['ae_adjusted_icer']:,.0f}/QALY")
    print(f"  Utility Reduction: {(1-ae_row['relative_utility'])*100:.2f}%")
    print(f"  QALYs Lost (5y): {loss_row['qalys_lost_5y']:.3f}")
    print(f"  Monetary Loss: ${loss_row['monetary_value_lost']:,.0f}")
    
    if row['original_icer'] != row['ae_adjusted_icer']:
        icer_chg = ((row['ae_adjusted_icer'] - row['original_icer']) / row['original_icer']) * 100
        print(f"  ICER Change due to AEs: {icer_chg:+.1f}%")
        
        if icer_chg > 10:
            print(f"  Impact: HIGH - Significant increase in ICER due to adverse events")
        elif icer_chg > 5:
            print(f"  Impact: MODERATE - Notable increase in ICER due to adverse events")
        else:
            print(f"  Impact: LOW - Minimal impact of adverse events on ICER")
    
    # Assess overall impact
    disutility_pct = ae_row['total_disutility'] / ae_row['base_utility'] * 100
    if disutility_pct > 15:
        print(f"  Overall AE Burden: HIGH")
    elif disutility_pct > 7:
        print(f"  Overall AE Burden: MODERATE")
    else:
        print(f"  Overall AE Burden: LOW")

# Rank strategies by AE-adjusted ICER
valid_strategies = ae_effect_df[ae_effect_df['ae_adjusted_icer'] != np.inf]
ranked_strategies = valid_strategies.sort_values('ae_adjusted_icer')
print(f"\n\nSTRATEGY RANKING (AE-ADJUSTED):")
print("="*30)
for i, (_, row) in enumerate(ranked_strategies.iterrows()):
    print(f"  {i+1}. {row['strategy']}: ${row['ae_adjusted_icer']:,.0f}/QALY")

## Next Steps

1. Integrate with real adverse event data from trials
2. Consider long-term sequelae of adverse events
3. Evaluate preventive measures and mitigation strategies
4. Model patient preferences for different AE profiles